<a href="https://colab.research.google.com/github/livio-24/test-deduplication/blob/main/word2vec_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [218]:
import os

configuration = 1   # possible values are: 1, 2, 3

###
# configuration=1 --> P1: tokenization, stop word removal
# configuration=2 --> P2: tokenization, stop word removal, lemmatization
# configuration=3 --> P3: tokenization, stop word removal, lemmatization, keyphrase extraction
###

folder="/content/drive/MyDrive/configuration_"+str(configuration)

if configuration == 1:
    lemmatization=False
    bigram=False
elif configuration == 2:
    lemmatization=True
    bigram=False
elif configuration == 3:
    lemmatization=True
    bigram=True

if not os.path.exists(folder):
    os.makedirs(folder)
if not os.path.exists(folder+"/models"):
    os.makedirs(folder+"/models")
if not os.path.exists(folder+"/simple"):
    os.makedirs(folder+"/simple")
if not os.path.exists(folder+"/tfidf"):
    os.makedirs(folder+"/tfidf")
if not os.path.exists(folder+"/simple/csv"):
    os.makedirs(folder+"/simple/csv")
if not os.path.exists(folder+"/tfidf/csv"):
    os.makedirs(folder+"/tfidf/csv")

In [ ]:
!pip install multi_rake

In [3]:
import pandas as pd
import numpy as np
import ast
import gensim, nltk, re
import spacy
from gensim.models.doc2vec import TaggedDocument
from gensim.models import KeyedVectors
import logging
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
#from multi_rake import Rake
import warnings
warnings.filterwarnings(action='once')

In [113]:
#data = get_dataframe_from_table(conn, 'news')
data = pd.read_csv('/content/drive/MyDrive/tesi/crime_news.csv')
#data = pd.read_csv(folder+"/dataset.csv")
data

,url,title,description,text,date_publication,time_publication,newspaper,date_event,tags
0,https://notizie.virgilio.it/intossicazione-ali...,Intossicazione alimentare nelle Rsa di Firenze...,Allarme intossicazione alimentare nelle Rsa di...,Allarme intossicazione alimentare nella Rsa Vi...,2025-02-13,17:11:44,Virgilio Notizie,2025-02-13,"{FIRENZE,TOSCANA}"
1,https://notizie.virgilio.it/neonata-trovata-mo...,"Neonata trovata morta in culla a Lucca, la mad...",Neonata trovata morta a Lucca nella sua culla:...,Proseguono le indagini sul caso della neonata ...,2025-02-12,11:56:37,Virgilio Notizie,2025-02-07,"{LUCCA,TOSCANA}"
2,https://notizie.virgilio.it/studente-si-accasc...,Studente si accascia a scuola a Cremona durant...,Uno studente di 14 anni è morto dopo aver accu...,Si è accasciato durante l’ora di ginnastica ed...,2025-02-10,19:10:22,Virgilio Notizie,2025-02-10,"{CREMONA,LOMBARDIA}"
3,https://notizie.virgilio.it/omicidio-baby-sitt...,Omicidio baby sitter Jhoanna Nataly Quintanill...,Continuano le indagini sull'omicidio della bab...,Ha ucciso la compagna Jhoanna Nataly Quintanil...,2025-02-10,18:14:14,Virgilio Notizie,2025-02-09,{}
4,https://notizie.virgilio.it/nuova-perizia-psic...,Nuova perizia psichiatrica per Alessia Pifferi...,La Corte d'Assise d'Appello ha disposto una nu...,Nuova perizia psichiatrica. È la decisione dei...,2025-02-10,16:55:31,Virgilio Notizie,2025-02-10,"{""ALESSIA PIFFERI""}"
...,...,...,...,...,...,...,...,...,...
4443,https://notizie.virgilio.it/ugento-21enne-colp...,"Ruffano, 21enne colpito con un pugno al volto ...","Il 21enne, colpito al volto con un pugno, è ri...","Un 21enne di Ugento, cittadina nella provincia...",2023-01-02,13:53:50,Virgilio Notizie,NaN,{}
4444,https://notizie.virgilio.it/il-giallo-di-termi...,Il giallo di Termini: turista israeliana accol...,La sera dell'ultimo dell'anno una turista isra...,Il giallo di Termini: la sera dell’ultimo dell...,2023-01-02,07:45:27,Virgilio Notizie,2023-01-02,"{ASIA,ISRAELE,LAZIO,ROMA}"
4445,https://notizie.virgilio.it/sicilia-giovane-uc...,"Vittoria, giovane ucciso a sprangate per aver ...","La vicenda è accaduta a Vittoria (Ragusa), dov...",Quella che doveva essere una serata di festa è...,2023-01-01,19:19:11,Virgilio Notizie,2023-01-01,{}
4446,https://notizie.virgilio.it/milano-25enne-feri...,"Milano, 25enne ferito da un colpo di pistola a...",Notte di sangue a Milano durante Capodanno: un...,Capodanno non è stata una serata dai potenzial...,2023-01-01,10:36:56,Virgilio Notizie,2023-01-01,"{LOMBARDIA,MILANO}"


In [114]:
print(data.isnull().sum())


url                   0
title                 0
description           5
text                  1
date_publication      0
time_publication      0
newspaper             0
date_event          620
tags                  0
dtype: int64


In [115]:
# Drop rows with null values in the 'text' column
data = data.dropna(subset=['text', 'description'])

In [ ]:
#data['text'] = data['text'].astype(str)

<ipython-input-10-e9083f2be5ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].astype(str)


In [ ]:
#data = data[~data['text'].str.startswith('Pubblicato: ')]

In [116]:
data.shape

(4442, 9)

In [117]:
data['newspaper'].value_counts()

,count
newspaper,
Virgilio Notizie,4442


# 3. Preprocessing

The preprocessing step includes lemmatization, stop words removal and tokenization

In [9]:
# Run this to download the file "italian.pickle"
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


True

In [10]:
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

### a. Without lemmatization

In [11]:
def review_to_wordlist(review, remove_stopwords=False):
    """
    Convert a review to a list of words. Removal of stop words is optional.
    """
    # remove non-letters
    review_text = (re.sub("[^a-zA-Z]"," ", review)).lower()

    # convert to lower case and split at whitespace
    words = review_text.split()

    # remove stop words (false by default)
    if remove_stopwords:
        stops = set(nltk.corpus.stopwords.words('italian'))
        words = [w for w in words if not w in stops]

    return words

In [12]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    """
    Split review into list of sentences where each sentence is a list of words.
    Removal of stop words is optional.
    """
    # use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())

    # each sentence is furthermore split into words
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            sentences += review_to_wordlist(raw_sentence, remove_stopwords)

    return sentences

In [ ]:
#data = data[['url', 'title', 'description','text', 'newspaper', 'date_publication', 'time_publication']].copy()

In [13]:
nltk.download('stopwords')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
%%time

#if lemmatization==False and bigram==False:
if configuration == 1:
    data.insert(3, 'preprocessed', data['text'].apply(lambda article: review_to_sentences(article, tokenizer, remove_stopwords=True)) )

CPU times: user 12.8 s, sys: 1.4 s, total: 14.2 s
Wall time: 14.8 s


### b. With Lemmatization

In [105]:
class DocPreprocess(object):

    def __init__(self, nlp, stop_words, docs, #labels,
                     build_bi=False, min_count=3, threshold=5,
                     allowed_postags=['ADV', 'VERB', 'ADJ', 'NOUN', 'PROPN', 'NUM']):

        self.nlp = nlp
        self.stop_words = stop_words
        self.docs = docs
        # self.labels = labels
        self.doc_ids = np.arange(len(docs))
        self.simple_doc_tokens = [gensim.utils.simple_preprocess(doc, deacc=True) for doc in self.docs]

        if build_bi:
            self.bi_detector = self.build_bi_detect(self.simple_doc_tokens, min_count=min_count, threshold=threshold)
            self.new_docs = self.make_bigram_doc(self.bi_detector, self.simple_doc_tokens)

            rake = Rake(min_chars=3,
                    max_words=3,
                    min_freq=2,
                    language_code='it',
                    stopwords=stop_words)
                    #punctuations=',;.:-_()\'!?"')
            self.keyphrase = self.docs.apply(lambda text: rake.apply(text, text_for_stopwords=None))
            self.new_keyphrases = [self.make_keyphrase_doc(doc) for doc in range(len(self.new_docs))]
        else:
            self.new_docs = self.make_simple_doc(self.simple_doc_tokens)
        self.doc_words = [self.lemmatize(doc, allowed_postags=allowed_postags) for doc in self.new_docs]
        self.tagdocs = [TaggedDocument(words=words, tags=[tag]) for words, tag in zip(self.doc_words, self.doc_ids)]

        if build_bi:
            [self.doc_words[doc].extend(self.new_keyphrases[doc]) for doc in range(len(self.new_docs))]

    def build_bi_detect(self, simple_doc_tokens, min_count, threshold):
        # nostopword_doc_tokens = [token for token in simple_doc_tokens if (token not in self.stop_words)]
        nostopword_doc_tokens = []
        for array in simple_doc_tokens:
            new_array=[]
            for element in array:
                if element not in self.stop_words:
                    new_array.append(element)
            nostopword_doc_tokens.append(new_array)

        bi_ = gensim.models.phrases.Phrases(nostopword_doc_tokens, min_count=min_count, threshold=threshold, scoring='default')
        bi_detector = gensim.models.phrases.Phraser(bi_)  # wrapper enhance efficiency
        return bi_detector

    def make_keyphrase_doc(self, doc):
        keyphrases = []
        keys = ""
        for k in self.keyphrase[doc]:
            keys = keys + " " + k[0].replace(' ', '_')  # concatenate back to a sentence
        keyphrases.append(keys)
        return keyphrases

    def make_bigram_doc(self, bi_detector, simple_doc_tokens):
        #nostopword_doc_tokens = [token for token in simple_doc_tokens if (token not in self.stop_words)]
        bi_doc_tokens = [bi_detector[doc_tokens] for doc_tokens in simple_doc_tokens]
        bi_docs = []
        for bi_tokens in bi_doc_tokens:
            bi_doc = " ".join(bi_tokens)  # concatenate back to a sentence
            bi_docs.append(bi_doc)
        return bi_docs

    def make_simple_doc(self, simple_doc_tokens):
        simple_docs = []
        for doc_tokens in simple_doc_tokens:
            simple = " ".join(doc_tokens)  # concatenate back to a sentence
            simple_docs.append(simple)
        return simple_docs

    def lemmatize(self, doc, allowed_postags):
        doc = self.nlp(doc)
        tokens = [token.lemma_ for token in doc if (token.text not in self.stop_words)]
                  #(token.pos_ in allowed_postags) and (token.text not in self.stop_words)]
        return tokens

In [106]:
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [107]:
# follow the instructions at https://spacy.io/usage for downloading

nlp = spacy.load('it_core_news_sm')
stop_words = nltk.corpus.stopwords.words('italian')

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [118]:
%%time

if configuration == 2:
    all_docs = DocPreprocess(nlp, stop_words, data['text'])
    data.insert(3, "preprocessed", all_docs.doc_words, True)

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


CPU times: user 3min 53s, sys: 727 ms, total: 3min 53s
Wall time: 3min 56s


### c. Keyphrase extraction

In [ ]:
%%time

if configuration == 3:
    all_docs = DocPreprocess(nlp, stop_words, data['text'], True)
    data.insert(3, "preprocessed", all_docs.doc_words, True)

In [126]:
data.preprocessed[0]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['allarme',
 'intossicazione',
 'alimentare',
 'rsa',
 'villa',
 'desiderio',
 'settignare',
 'frazione',
 'firenza',
 'morire',
 'ospite',
 'dopo',
 'avere',
 'accusare',
 'malore',
 'seguito',
 'pasto',
 'consumare',
 'struttura',
 'vittima',
 'totale',
 'pero',
 'primo',
 'aggiungere',
 'infatti',
 'ospite',
 'altro',
 'rsa',
 'struttura',
 'coinvolgere',
 'infatti',
 'quarto',
 'ospite',
 'grave',
 'condizione',
 'pronto',
 'soccorso',
 'essere',
 'diagnosticare',
 'grave',
 'sindrome',
 'medico',
 'dopo',
 'avere',
 'provare',
 'salvare li',
 'potere',
 'fare',
 'altro',
 'constatare',
 'decesso',
 'base',
 'primo',
 'ricostruzione',
 'anziano',
 'accusare',
 'malore',
 'termine',
 'pasto',
 'rsa',
 'terzo',
 'perono',
 'ospite',
 'altro',
 'rsa',
 'morire',
 'invece',
 'giovedi',
 'febbraio',
 'quarto',
 'invece',
 'ricoverare',
 'grave',
 'condizione',
 'ospedale',
 'careggio',
 'firenza',
 'giorno',
 'scorso',
 'nas',
 'carabiniere',
 'personale',
 'ufficio',
 'igiene',
 'asl',

Delete those articles whose word list is empty

In [120]:
indexes = []
for index, sent in enumerate(data['preprocessed']):
    if len(sent) == 0:
        indexes.append(index)

In [121]:
data = data.drop(indexes, axis=0)
data.shape

(4442, 10)

In [122]:
data = data.reset_index(drop=True)

In [123]:
data

,url,title,description,preprocessed,text,date_publication,time_publication,newspaper,date_event,tags
0,https://notizie.virgilio.it/intossicazione-ali...,Intossicazione alimentare nelle Rsa di Firenze...,Allarme intossicazione alimentare nelle Rsa di...,"[allarme, intossicazione, alimentare, rsa, vil...",Allarme intossicazione alimentare nella Rsa Vi...,2025-02-13,17:11:44,Virgilio Notizie,2025-02-13,"{FIRENZE,TOSCANA}"
1,https://notizie.virgilio.it/neonata-trovata-mo...,"Neonata trovata morta in culla a Lucca, la mad...",Neonata trovata morta a Lucca nella sua culla:...,"[proseguire, indagine, caso, neonata, trovato,...",Proseguono le indagini sul caso della neonata ...,2025-02-12,11:56:37,Virgilio Notizie,2025-02-07,"{LUCCA,TOSCANA}"
2,https://notizie.virgilio.it/studente-si-accasc...,Studente si accascia a scuola a Cremona durant...,Uno studente di 14 anni è morto dopo aver accu...,"[accasciare, durante, ora, ginnastica, morire,...",Si è accasciato durante l’ora di ginnastica ed...,2025-02-10,19:10:22,Virgilio Notizie,2025-02-10,"{CREMONA,LOMBARDIA}"
3,https://notizie.virgilio.it/omicidio-baby-sitt...,Omicidio baby sitter Jhoanna Nataly Quintanill...,Continuano le indagini sull'omicidio della bab...,"[uccidere, compagna, Jhoanna, nataly, quintani...",Ha ucciso la compagna Jhoanna Nataly Quintanil...,2025-02-10,18:14:14,Virgilio Notizie,2025-02-09,{}
4,https://notizie.virgilio.it/nuova-perizia-psic...,Nuova perizia psichiatrica per Alessia Pifferi...,La Corte d'Assise d'Appello ha disposto una nu...,"[nuovo, perizia, psichiatrico, decisione, giud...",Nuova perizia psichiatrica. È la decisione dei...,2025-02-10,16:55:31,Virgilio Notizie,2025-02-10,"{""ALESSIA PIFFERI""}"
...,...,...,...,...,...,...,...,...,...,...
4437,https://notizie.virgilio.it/ugento-21enne-colp...,"Ruffano, 21enne colpito con un pugno al volto ...","Il 21enne, colpito al volto con un pugno, è ri...","[enne, ugento, cittadino, provincia, leccia, r...","Un 21enne di Ugento, cittadina nella provincia...",2023-01-02,13:53:50,Virgilio Notizie,NaN,{}
4438,https://notizie.virgilio.it/il-giallo-di-termi...,Il giallo di Termini: turista israeliana accol...,La sera dell'ultimo dell'anno una turista isra...,"[giallo, termine, sera, ultimo, anno, turista,...",Il giallo di Termini: la sera dell’ultimo dell...,2023-01-02,07:45:27,Virgilio Notizie,2023-01-02,"{ASIA,ISRAELE,LAZIO,ROMA}"
4439,https://notizie.virgilio.it/sicilia-giovane-uc...,"Vittoria, giovane ucciso a sprangate per aver ...","La vicenda è accaduta a Vittoria (Ragusa), dov...","[dovere, essere, serata, festa, finire, peggio...",Quella che doveva essere una serata di festa è...,2023-01-01,19:19:11,Virgilio Notizie,2023-01-01,{}
4440,https://notizie.virgilio.it/milano-25enne-feri...,"Milano, 25enne ferito da un colpo di pistola a...",Notte di sangue a Milano durante Capodanno: un...,"[capodanno, essere, serata, potenziale, risvol...",Capodanno non è stata una serata dai potenzial...,2023-01-01,10:36:56,Virgilio Notizie,2023-01-01,"{LOMBARDIA,MILANO}"


In [127]:
data.to_csv(folder+"/dataset.csv", index=False)

# 4. Get a Word2Vec Model

### a. Use a pre-trained model

In [ ]:
data = pd.read_csv(folder+"/dataset.csv")

In [ ]:
%%time

# Download model from https://mlunicampania.gitlab.io/italian-word2vec/

pretrained_word_vectors = KeyedVectors.load("W2V.kv", mmap='r+')
#pretrained_word_vectors = KeyedVectors.load("model M3.kv", mmap='r+')

#pretrained_vocabs = pretrained_word_vectors.index_to_key
pretrained_vectors = pretrained_word_vectors.vectors

### b. Train a model from scratch

In [128]:
%%time

from gensim.models import Word2Vec

def train_model_from_scratch():
    w2v_model = Word2Vec(sg=1,
                     min_count=20,
                     window=10,
                     vector_size=300,
                     sample=1e-3,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=7)
    w2v_model.build_vocab(data['preprocessed'], progress_per=10000)
    w2v_model.train(data['preprocessed'], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    model_name = folder+"/models/from_scratch.kv"
    word_vectors = w2v_model.wv
    word_vectors.save(model_name)

train_model_from_scratch()
new_word_vectors = KeyedVectors.load(folder+"/models/from_scratch.kv", mmap='r+')
#new_vocabs = new_word_vectors.index_to_key
new_vectors = new_word_vectors.vectors

CPU times: user 37min 9s, sys: 3.94 s, total: 37min 13s
Wall time: 22min 34s


### c. Retrain a pre-trained Model

In [ ]:
%%time

from gensim.models import Word2Vec, KeyedVectors

def retrain_pretrained_model():
    sentences = data['preprocessed'].values.tolist()
    w2v_model = Word2Vec(sg=1,
                    min_count=20,
                    window=10,
                    size=300,
                    sample=1e-3,
                    negative=20,
                    workers=7)
    w2v_model.build_vocab(sentences)
    w2v_model.intersect_word2vec_format('W2V.bin', binary=True, lockf=1.0)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs, report_delay=1)
    model_name = folder+"/models/retrained.kv"
    word_vectors = w2v_model.wv
    word_vectors.save(model_name)

retrain_pretrained_model()
retrained_word_vectors = KeyedVectors.load(folder+"/models/retrained.kv", mmap='r+')
#retrained_vocabs = retrained_word_vectors.index_to_key
retrained_vectors = retrained_word_vectors.vectors

# 5. Mean Word Embedding

### a. Simple Mean

In [129]:
class MeanEmbeddingVectorizer(object):

    def __init__(self, word_model):
        self.word_model = word_model
        self.vector_size = word_model.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """
        mean = []
        for word in sent:
            if word in self.word_model.key_to_index:
                mean.append(self.word_model.get_vector(word))


        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

### b. Mean weighted by TF-IDF score

In [25]:
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param
        pre_processed_docs: list of docs, which are tokenized
        :return:
        self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(lambda: max_idf, [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self


    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector


    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.key_to_index:
                mean.append(self.word_model.get_vector(word) * self.word_idf_weight[word])  # idf weighted

        if len(mean) == 0:  # empty words
            # If a text is empty, return a vector of zeros.
            logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

### Compute word embeddings and store them on csv files

In [130]:
def vectors_to_csv(word_vectors, mean, lemmatization, bigram, filename):
    if mean=='simple':
        vectorizer = MeanEmbeddingVectorizer(word_vectors)
        vectorizer.fit()
        features = vectorizer.transform(data['preprocessed'])
    elif mean=='tfidf':
        vectorizer = TfidfEmbeddingVectorizer(word_vectors)
        vectorizer.fit(data['preprocessed'])
        features = vectorizer.transform(data['preprocessed'])

    vectors = pd.DataFrame(data=features)
    vectors.insert(0, 'url', data['url'].values)
    vectors.insert(1, 'title', data['title'].values)
    vectors.insert(2, 'description', data['description'].values)
    vectors.insert(3, 'text', data['text'].values)
    vectors.insert(4, 'date_publication', data['date_publication'].values)
    vectors.insert(5, 'time_publication', data['time_publication'].values)
    vectors.insert(6, 'newspaper', data['newspaper'].values)
    vectors.insert(7, 'date_event', data['date_event'].values)
    vectors.insert(8, 'tags', data['tags'].values)
    vectors.insert(9, 'preprocessed', data['preprocessed'].values)
    #vectors.insert(7, 'target', data['tag'].values)
    vectors = vectors.reset_index(drop=True)

    vectors.to_csv(folder+'/'+mean+'/csv/'+filename+'_vectors.csv', index=False)

In [ ]:
%%time

vectors_to_csv(pretrained_word_vectors, 'simple', lemmatization, bigram, 'pretrained')

In [ ]:
%%time

vectors_to_csv(pretrained_word_vectors, 'tfidf', lemmatization, bigram, 'pretrained')

In [131]:
%%time

vectors_to_csv(new_word_vectors, 'simple', lemmatization, bigram, 'new')

CPU times: user 6.26 s, sys: 121 ms, total: 6.38 s
Wall time: 11.1 s


In [ ]:
%%time

vectors_to_csv(new_word_vectors, 'tfidf', lemmatization, bigram, 'new')

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


CPU times: user 11.9 s, sys: 218 ms, total: 12.1 s
Wall time: 14.7 s


In [ ]:
%%time

vectors_to_csv(retrained_word_vectors, 'simple', lemmatization, bigram, 'retrained')

Wall time: 41.2 s


In [ ]:
%%time

vectors_to_csv(retrained_word_vectors, 'tfidf', lemmatization, bigram, 'retrained')

Wall time: 1min 32s


# DOCUMENTS SIMILARITY CALCULATION


In [219]:
from sklearn.metrics.pairwise import cosine_similarity

In [220]:
emb = pd.read_csv(folder + '/simple/csv/new_vectors.csv')

In [221]:
link = pd.read_csv('/content/drive/MyDrive/tesi/crime_news_link2.csv')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [222]:
link.head()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,url
0,1,https://notizie.virgilio.it/intossicazione-ali...
1,2,https://notizie.virgilio.it/neonata-trovata-mo...
2,3,https://notizie.virgilio.it/studente-si-accasc...
3,4,https://notizie.virgilio.it/omicidio-baby-sitt...
4,5,https://notizie.virgilio.it/nuova-perizia-psic...


In [223]:
# prompt: merge link end emb on url

# Merge the two dataframes based on the 'url' column
merged_df = pd.merge(link, emb, on='url').sort_values(by='id', ascending = True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [224]:
merged_df

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,url,title,description,text,date_publication,time_publication,newspaper,date_event,tags,...,290,291,292,293,294,295,296,297,298,299
0,1,https://notizie.virgilio.it/intossicazione-ali...,Intossicazione alimentare nelle Rsa di Firenze...,Allarme intossicazione alimentare nelle Rsa di...,Allarme intossicazione alimentare nella Rsa Vi...,2025-02-13,17:11:44,Virgilio Notizie,2025-02-13,"{FIRENZE,TOSCANA}",...,-0.103506,0.050266,0.007896,-0.022895,0.131707,0.107489,0.055068,-0.032545,-0.067181,-0.107123
1,2,https://notizie.virgilio.it/neonata-trovata-mo...,"Neonata trovata morta in culla a Lucca, la mad...",Neonata trovata morta a Lucca nella sua culla:...,Proseguono le indagini sul caso della neonata ...,2025-02-12,11:56:37,Virgilio Notizie,2025-02-07,"{LUCCA,TOSCANA}",...,-0.022941,0.036573,-0.027107,-0.016407,0.140188,0.027849,0.082890,-0.036635,-0.100779,-0.080624
2,3,https://notizie.virgilio.it/studente-si-accasc...,Studente si accascia a scuola a Cremona durant...,Uno studente di 14 anni è morto dopo aver accu...,Si è accasciato durante l’ora di ginnastica ed...,2025-02-10,19:10:22,Virgilio Notizie,2025-02-10,"{CREMONA,LOMBARDIA}",...,-0.007744,0.045109,0.038990,0.001926,0.169427,-0.006367,0.034582,0.026516,-0.135609,-0.069749
3,4,https://notizie.virgilio.it/omicidio-baby-sitt...,Omicidio baby sitter Jhoanna Nataly Quintanill...,Continuano le indagini sull'omicidio della bab...,Ha ucciso la compagna Jhoanna Nataly Quintanil...,2025-02-10,18:14:14,Virgilio Notizie,2025-02-09,{},...,-0.093775,0.081117,-0.000160,-0.029348,0.113535,-0.003026,0.073907,-0.015893,-0.072754,0.001166
4,5,https://notizie.virgilio.it/nuova-perizia-psic...,Nuova perizia psichiatrica per Alessia Pifferi...,La Corte d'Assise d'Appello ha disposto una nu...,Nuova perizia psichiatrica. È la decisione dei...,2025-02-10,16:55:31,Virgilio Notizie,2025-02-10,"{""ALESSIA PIFFERI""}",...,-0.200609,0.000507,-0.143178,0.049970,0.183182,-0.016648,0.034021,0.095144,-0.056209,-0.241398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,4490,https://notizie.virgilio.it/ugento-21enne-colp...,"Ruffano, 21enne colpito con un pugno al volto ...","Il 21enne, colpito al volto con un pugno, è ri...","Un 21enne di Ugento, cittadina nella provincia...",2023-01-02,13:53:50,Virgilio Notizie,NaN,{},...,-0.053029,0.100920,0.119418,0.078088,0.198123,0.073691,0.089039,0.020083,-0.096343,-0.011547
4438,4491,https://notizie.virgilio.it/il-giallo-di-termi...,Il giallo di Termini: turista israeliana accol...,La sera dell'ultimo dell'anno una turista isra...,Il giallo di Termini: la sera dell’ultimo dell...,2023-01-02,07:45:27,Virgilio Notizie,2023-01-02,"{ASIA,ISRAELE,LAZIO,ROMA}",...,-0.060543,0.133646,0.019622,-0.007753,0.182342,0.007699,0.033570,-0.000701,-0.078586,0.033837
4439,4492,https://notizie.virgilio.it/sicilia-giovane-uc...,"Vittoria, giovane ucciso a sprangate per aver ...","La vicenda è accaduta a Vittoria (Ragusa), dov...",Quella che doveva essere una serata di festa è...,2023-01-01,19:19:11,Virgilio Notizie,2023-01-01,{},...,0.001661,0.049631,0.069184,0.035365,0.151722,0.059490,0.038310,-0.038127,-0.070212,0.021946
4440,4493,https://notizie.virgilio.it/milano-25enne-feri...,"Milano, 25enne ferito da un colpo di pistola a...",Notte di sangue a Milano durante Capodanno: un...,Capodanno non è stata una serata dai potenzial...,2023-01-01,10:36:56,Virgilio Notizie,2023-01-01,"{LOMBARDIA,MILANO}",...,-0.056928,0.091631,0.065716,0.048772,0.140836,0.052509,0.037998,-0.005922,-0.039756,0.064177


In [225]:
#emb = emb.sort_values(by='date_publication', ascending = False).reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [226]:
emb_array = merged_df.iloc[:,11:].values

In [227]:
emb_array

array([[ 0.00850542,  0.21335404, -0.11648665, ..., -0.03254506,
        -0.06718123, -0.1071232 ],
       [ 0.06812244,  0.18276788, -0.09115961, ..., -0.03663479,
        -0.10077908, -0.08062424],
       [-0.06124685,  0.15389873, -0.08339457, ...,  0.02651567,
        -0.13560888, -0.06974892],
       ...,
       [ 0.03537614,  0.08952075, -0.09520093, ..., -0.03812669,
        -0.07021192,  0.02194599],
       [ 0.05378428,  0.12254562, -0.13913096, ..., -0.0059225 ,
        -0.03975626,  0.06417695],
       [ 0.08853441,  0.10192463, -0.12929016, ..., -0.00983805,
        -0.06007679,  0.04835907]])

In [228]:
similarity_matrix = cosine_similarity(emb_array)

In [274]:
threshold = 0.93
num_docs = 100 #len(emb_array)

near_duplicates = []

# Iterate through the upper triangle of the matrix to avoid duplicate pairs
for i in range(num_docs):
    for j in range(i + 1, num_docs):  # Avoid self-comparison
        if similarity_matrix[i, j] > threshold:
            near_duplicates.append((i , j, similarity_matrix[i, j]))

# Convert to DataFrame for better visualization
near_duplicates_df = pd.DataFrame(near_duplicates, columns=["Doc1", "Doc2", "Similarity"])
print(near_duplicates_df)

    Doc1  Doc2  Similarity
0      5    22    0.931817
1      9    11    0.939699
2     10    89    0.936049
3     14    16    0.945801
4     18    19    0.949455
5     27    84    0.931714
6     30    31    0.946648
7     33    44    0.954366
8     37    86    0.936354
9     39    45    0.953190
10    43    59    0.946370
11    62    77    0.931861
12    63    97    0.952774
13    69    78    0.957905
14    70    76    0.932474
15    70    79    0.956735
16    76    79    0.949737
17    82    88    0.944863


In [275]:
print(emb['text'][54])
print(emb['text'][2342])

Le indagini sul Mostro di Firenze potrebbero essere presto riaperte. L’istanza, già precedentemente presentata alla Procura, si arricchisce di nuovi elementi fra i quali il possibile coinvolgimento di un ex dipendente pubblico di Firenze, ormai morto da anni. Ma ai pm si chiede anche di vagliare alcune testimonianze. La richiesta di riapertura delle indagini è stata presentata, su incarico di un parente delle vittime, dall’avvocato Alessio Tranfa e dal consulente Paolo Cochi. L’identikit si riferisce a un uomo indicato da Susanna Cambi, uccisa insieme al fidanzato Stefano Baldi, nei campi delle Bartoline. La donna si era confidata con un’amica raccontando di essere pedinata da un individuo con quelle fattezze alla guida di una “Alfa Romeo rossa”. Testimoni dell’epoca concordarono nel riferirsi a un uomo sui 50 anni, di corporatura robusta e di un’altezza stimata sul metro e 80. Qualcuno sul momento non ci fece caso, scambiandolo per un guardone. Un testimone raccontò che l’uomo guidava

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [276]:
# number of duplicates news

unique_doc1_elements = near_duplicates_df["Doc1"].unique()
unique_doc1_elements.size

17

In [277]:
#near_duplicates_df.to_csv(folder+'/simple/csv/near_duplicates.csv', index=False)

In [278]:
# prompt: groupby Doc1 and create a dataframe having the doc id and the list of Doc2 grouped

import pandas as pd

# Assuming 'near_duplicates_df' is your DataFrame from the previous code
# Replace 'your_file.csv' with the actual path to your CSV file if needed.
# near_duplicates_df = pd.read_csv('your_file.csv')

grouped_df = near_duplicates_df.groupby('Doc1')['Doc2'].apply(list).reset_index()
grouped_df

,Doc1,Doc2
0,5,[22]
1,9,[11]
2,10,[89]
3,14,[16]
4,18,[19]
5,27,[84]
6,30,[31]
7,33,[44]
8,37,[86]
9,39,[45]


In [279]:
val_set = pd.read_csv('/content/drive/MyDrive/tesi/val_set2.csv')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [280]:
#val_set.to_csv('/content/drive/MyDrive/tesi/validation_set_duplication.csv', index=False)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [281]:
val_set

,id,url,title,description,text,date_publication,time_publication,newspaper,date_event,tags,is_dup
0,1,https://notizie.virgilio.it/intossicazione-ali...,Intossicazione alimentare nelle Rsa di Firenze...,Allarme intossicazione alimentare nelle Rsa di...,Allarme intossicazione alimentare nella Rsa Vi...,2025-02-13,17:11:44,Virgilio Notizie,2025-02-13,"{FIRENZE,TOSCANA}",0
1,2,https://notizie.virgilio.it/neonata-trovata-mo...,"Neonata trovata morta in culla a Lucca, la mad...",Neonata trovata morta a Lucca nella sua culla:...,Proseguono le indagini sul caso della neonata ...,2025-02-12,11:56:37,Virgilio Notizie,2025-02-07,"{LUCCA,TOSCANA}",0
2,3,https://notizie.virgilio.it/studente-si-accasc...,Studente si accascia a scuola a Cremona durant...,Uno studente di 14 anni è morto dopo aver accu...,Si è accasciato durante l’ora di ginnastica ed...,2025-02-10,19:10:22,Virgilio Notizie,2025-02-10,"{CREMONA,LOMBARDIA}",0
3,4,https://notizie.virgilio.it/omicidio-baby-sitt...,Omicidio baby sitter Jhoanna Nataly Quintanill...,Continuano le indagini sull'omicidio della bab...,Ha ucciso la compagna Jhoanna Nataly Quintanil...,2025-02-10,18:14:14,Virgilio Notizie,2025-02-09,{},1
4,5,https://notizie.virgilio.it/nuova-perizia-psic...,Nuova perizia psichiatrica per Alessia Pifferi...,La Corte d'Assise d'Appello ha disposto una nu...,Nuova perizia psichiatrica. È la decisione dei...,2025-02-10,16:55:31,Virgilio Notizie,2025-02-10,"{""ALESSIA PIFFERI""}",1
...,...,...,...,...,...,...,...,...,...,...,...
95,96,https://notizie.virgilio.it/rapina-nella-villa...,Rapina nella villa di Maria Sole Agnelli a Rom...,Alcuni ladri hanno fatto irruzione nella casa ...,Una banda di rapinatori ha fatto irruzione nel...,2025-01-09,19:01:31,Virgilio Notizie,2025-01-09,"{LAZIO,ROMA}",0
96,97,https://notizie.virgilio.it/omicidio-di-kelly-...,"Omicidio di Kelly Egbon a Schio, 32enne morto ...","Confessato l'omicidio di Kelly Egbon, il 32enn...",Queen Enabuele ha confessato l’omicidio del fi...,2025-01-09,10:09:32,Virgilio Notizie,NaN,"{VENETO,VICENZA}",0
97,98,https://notizie.virgilio.it/neonato-trovato-mo...,"Neonato trovato morto a Bari in culla, autopsi...",L'autopsia del neonato trovato morto nella cul...,Il neonato trovato morto a Bari lo scorso 2 ge...,2025-01-08,23:46:17,Virgilio Notizie,2025-01-02,"{BARI,PUGLIA}",0
98,99,https://notizie.virgilio.it/giuseppe-barbaro-m...,Giuseppe Barbaro morto in ospedale a Palermo d...,"Giuseppe Barbaro, 76 anni, è morto all'ospedal...","Giuseppe Barbaro, 76 anni, è morto dopo 17 gio...",2025-01-08,17:54:05,Virgilio Notizie,2024-12-24,"{PALERMO,SICILIA}",0


In [282]:
#val_set["is_dup"] = val_set["Doc2"].apply(lambda x: 0 if x == '-1' else 1)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [283]:
#val_set[['id','is_dup']].to_csv('/content/drive/MyDrive/tesi/validation_set_duplication_ids.csv', index=False)

In [284]:
#val_set.to_csv('/content/drive/MyDrive/tesi/validation_set_duplication.csv', index=False)

In [285]:
val_set['dup_pred'] = 0  # Initialize with a default value

# Iterate through the grouped DataFrame
for index, row in grouped_df.iterrows():
    doc1_index = row['Doc1']

    val_set.loc[doc1_index, 'dup_pred'] = 1  # Convert the list to a string for storage

In [286]:
val_set[["id","is_dup","dup_pred"]].head(30)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,is_dup,dup_pred
0,1,0,0
1,2,0,0
2,3,0,0
3,4,1,0
4,5,1,0
5,6,0,1
6,7,0,0
7,8,0,0
8,9,0,0
9,10,1,1


In [287]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [288]:
y_true = val_set["is_dup"]
y_pred = val_set["dup_pred"]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.82
Precision: 0.65
Recall: 0.48
F1 Score: 0.55
